In [2]:
import xlrd # read file
from sentence_transformers import SentenceTransformer # create embeddings

# FAISS vector db
import faiss
import numpy as np
import json

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file_path = 'Статьи.xls'

# Read file

In [4]:
workbook = xlrd.open_workbook(file_path)
workbook

In [5]:
sheet = workbook.sheet_by_index(0)
sheet

Sheet  0:<Выгрузка из БЗ>

In [6]:
data = []

In [7]:
for row in range(1, sheet.nrows):
        short_topic_name = sheet.cell_value(row, 0)  # Column 0
        actual_knowledge = sheet.cell_value(row, 1)  # Column 1
        data.append({
            "short_topic_name": short_topic_name,
            "actual_knowledge": actual_knowledge
        })

In [8]:
data

[{'short_topic_name': 'Подписание контракта после отказа от заключения со стороны заказчика.',
  'actual_knowledge': 'После отказа от заключения контракта заказчик вместо отправления нового проекта продлил сроки подписания по неактуальному документу. Для решения данного вопроса вам необходимо связаться с заказчиком, для направления нового проекта контракта.\xa0'},
 {'short_topic_name': 'Участие в котировочной сессии', 'actual_knowledge': ''},
 {'short_topic_name': 'Страница контракта организации',
  'actual_knowledge': 'Страница контракта содержит следующие сведения:\n\xa0\n\xa0\n\n\xa0\n\xa0\n\xa0\n\nРеестровый номер контракта;\nНаименование закупки;\nЗаказчик/Покупатель;\nИНН Заказчика;\nДата заключения. Данное поле становится заполненным после подписания контракта/договора последней стороной (заказчиком);\nПериод действия контракта. Дата предполагаемые и поля доступны для редактирования;\nПредмет контракта;\nМесто заключения контракта;\nИдентификационный код закупки (ИКЗ);\nИсточник

# Create embeddings

In [9]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [10]:
texts = [entry["actual_knowledge"] for entry in data]
texts

['После отказа от заключения контракта заказчик вместо отправления нового проекта продлил сроки подписания по неактуальному документу. Для решения данного вопроса вам необходимо связаться с заказчиком, для направления нового проекта контракта.\xa0',
 '',
 'Страница контракта содержит следующие сведения:\n\xa0\n\xa0\n\n\xa0\n\xa0\n\xa0\n\nРеестровый номер контракта;\nНаименование закупки;\nЗаказчик/Покупатель;\nИНН Заказчика;\nДата заключения. Данное поле становится заполненным после подписания контракта/договора последней стороной (заказчиком);\nПериод действия контракта. Дата предполагаемые и поля доступны для редактирования;\nПредмет контракта;\nМесто заключения контракта;\nИдентификационный код закупки (ИКЗ);\nИсточник финансирования;\nЗакон, в соответствии с которым размещена закупка;\nСпособ размещения закупки (заказа)/определение поставщика;\nОснование заключения контракта;\nУсловия исполнения контракта (указывается «Обязательное электронное исполнение с использованием УПД» при э

In [11]:
embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
embeddings

Batches: 100%|██████████| 20/20 [00:18<00:00,  1.07it/s]


array([[-0.02566585,  0.24214806, -0.20930636, ..., -0.03561375,
         0.02720529,  0.07469096],
       [ 0.22691739,  0.08178417,  0.02354266, ..., -0.09982924,
        -0.03107593,  0.07418393],
       [-0.14252996,  0.05594344, -0.24717113, ..., -0.04219484,
        -0.00075015,  0.06846327],
       ...,
       [ 0.08895024,  0.01456913,  0.03632306, ..., -0.19645022,
        -0.00539186,  0.0925234 ],
       [-0.0266958 ,  0.05558772,  0.03846874, ...,  0.04246728,
         0.05012774,  0.05067867],
       [-0.09430532,  0.00596469, -0.04213819, ...,  0.02893157,
         0.01251571,  0.18047297]], dtype=float32)

In [12]:
dimension = embeddings.shape[1]
dimension

384

In [13]:
index = faiss.IndexFlatL2(dimension)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x156fc06f0> >

In [14]:
index.add(embeddings.astype('float32'))
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x156fc06f0> >

In [15]:
faiss.write_index(index, "knowledge_base_index.faiss")

In [22]:
metadata = [{"short_topic_name": entry["short_topic_name"], "actual_knowledge": entry["actual_knowledge"]} for entry in data]

In [24]:
with open("knowledge_base_metadata.json", "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)